In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

froot = './data/k562_chr16_updated'

df = pd.read_csv(froot + ".csv")

column_names = np.array(df.columns)
feature_names = column_names[6:-2]
num_features = len(feature_names)
print(feature_names)
num_samples = df.shape[0]

#Y_ji is a list of samples containing lists of their feature values
    # [   
    #   sample_1: [feat_1, feat_2,...,feat_n],
    #   sample_2: [feat_1, feat_2,...,feat_n],
    # ]

Y_ji = df.iloc[:, 6:-2].values

# process GLM simulated elongation rates
Z_ji = df['zeta'].values

model = nn.Linear(num_features, 1, bias=False, dtype=torch.float64)

model.weight.data[0] = torch.tensor([-0.0224536145637661, -0.094592589, -0.023815382, 0.030402922, -0.067234092, -0.032196914, -0.040911478, -0.018557168, -0.033545905, -0.051103287, -0.204434712, 0.015831043], dtype=torch.float64)

model.eval()

weights = model.weight.data.cpu().numpy()
combined = ', '.join([f'"{s}": {f}' for s, f in zip(feature_names, weights[0])])
print(combined)

['ctcf' 'h3k36me3' 'h3k4me1' 'h3k79me2' 'h3k9me1' 'h3k9me3' 'h4k20me1'
 'sj5' 'sj3' 'dms' 'wgbs' 'rpts']
"ctcf": -0.0224536145637661, "h3k36me3": -0.094592589, "h3k4me1": -0.023815382, "h3k79me2": 0.030402922, "h3k9me1": -0.067234092, "h3k9me3": -0.032196914, "h4k20me1": -0.040911478, "sj5": -0.018557168, "sj3": -0.033545905, "dms": -0.051103287, "wgbs": -0.204434712, "rpts": 0.015831043


In [8]:
with torch.no_grad():
    y_inputs = Y_ji
    rho_ji = model(torch.tensor(y_inputs, dtype=torch.float64))

    matmul_zeta = torch.exp(torch.matmul(torch.tensor(Y_ji, dtype=torch.float64), model.weight.data[0]))
    
    # convert log(Z) outputs to Z
    neural_net_zeta = torch.exp(rho_ji.squeeze())
    
    print("Neural Network Zeta:")
    print(neural_net_zeta.numpy())
    print("\n")

    print("Matrix Multiplication Zeta: Z_ji = e^(k * Y_ji)")
    print(matmul_zeta.numpy())
    print("\n")
    
    print("Ling Zeta:")
    print(Z_ji)

Neural Network Zeta:
[1.07487679 1.07400773 1.07313596 ... 0.37690437 0.37699961 0.37709801]


Matrix Multiplication Zeta: Z_ji = e^(k * Y_ji)
[1.07487679 1.07400773 1.07313596 ... 0.37690437 0.37699961 0.37709801]


Ling Zeta:
[1.07487679 1.07400773 1.07313596 ... 0.37690437 0.37699961 0.37709801]
